In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

ds = load_dataset("ai-forever/kinopoisk-sentiment-classification")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/104 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/37.1M [00:00<?, ?B/s]

validation.jsonl:   0%|          | 0.00/5.39M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/5.27M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10500 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 10500
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 1500
    })
})

In [4]:
ds['test'][0]

{'id': '0',
 'text': 'Шла на сеанс, не ожидая ничего сверхъестественного ни от Джессики Симпсон ни от самого фильма. И надо сказать, ушла из кинотеатра с приятным удивлением в душе. Надо же, как ей идут роли деревенских простушек в самом лучшем смысле этих слов. Сначала в «Придурках из Хаззарда», а теперь тут Джесси просто блеснула своим врожденным обаянием.\n\nПусть она далеко не самая хорошая актриса, но фильм получился лучше, чем я ожидала. Смотреть было приятно, смешно и весело. Мы с друзьями отправились домой в превосходном расположении духа, а мне даже за рулем по дороге вспоминалась та потешная сцена, где она с велосипедом вместе повалилась в строительную яму.\n\nТакое кино ни к чему не обязывает и в этом его прелесть. Оно не испортит никому настроения, разве что серьезным дядям из киноакадемии, которые раздают Оскары. Но ведь это не мы с вами :)\n\nА то, что «Блондинка с амбициями» лучше «Блондинки в шоколаде» можна сказать даже не смотрев ни то ни другое. Просто в актерском со

In [5]:
set(ds['test']['label_text'])

{'Bad', 'Good', 'Neutral'}

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [7]:
# Загрузка модели и токенизатора
tokenizer = AutoTokenizer.from_pretrained("DAMO-NLP-SG/zero-shot-classify-SSTuning-XLM-R")
model = AutoModelForSequenceClassification.from_pretrained("DAMO-NLP-SG/zero-shot-classify-SSTuning-XLM-R")

# Настройка устройства (использование GPU, если доступно)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device).eval()

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

In [8]:
# Маппинг классов для совместимости с моделью
label_map = {
    "Bad": "negative",
    "Good": "positive",
    "Neutral": "neutral"
}

In [9]:
# Тестовые данные
test_texts = ds['test']['text']
true_labels = ds['test']['label_text']
label_names = list(label_map.values())

In [10]:
# Функция предсказания
def predict(text):
    labels_for_model = [label + "." for label in label_names]
    labels_for_model += [tokenizer.pad_token] * (20 - len(labels_for_model))

    formatted_text = ' '.join([f"({chr(65 + i)}) {labels_for_model[i]}" for i in range(len(labels_for_model))]) + f" {tokenizer.sep_token} {text}"

    encoding = tokenizer([formatted_text], truncation=True, max_length=512, return_tensors='pt')
    item = {key: val.to(device) for key, val in encoding.items()}

    with torch.no_grad():
        logits = model(**item).logits[:, :len(label_names)]
        probs = torch.nn.functional.softmax(logits, dim=-1)

    prediction_idx = torch.argmax(probs, dim=-1).item()
    prediction = label_names[prediction_idx]
    probability = probs[0, prediction_idx].item()
    return prediction, probability

In [11]:
# Применение модели к тестовому набору
predictions = []
probabilities = []

for text in tqdm(test_texts, desc="Predicting"):
    pred_label, prob = predict(text)
    predictions.append(pred_label)
    probabilities.append(prob)

# Вычисление метрик
y_true = [label_map[label] for label in true_labels]  # Приведение к формату, ожидаемому моделью
y_pred = predictions

Predicting: 100%|██████████| 1500/1500 [00:50<00:00, 29.44it/s]


In [12]:
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted')
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Precision:", precision)
print("Recall:", recall)

Accuracy: 0.5326666666666666
F1 Score: 0.4937527502287174
Precision: 0.5172356691941274
Recall: 0.5326666666666666


In [13]:
import pandas as pd

In [14]:
# Создание DataFrame для сабмита
submission_df = pd.DataFrame({
    'text': test_texts,
    'true_label': true_labels,
    'predicted_label': predictions,
    'probability': probabilities
})



In [15]:
submission_df

,text,true_label,predicted_label,probability
0,"Шла на сеанс, не ожидая ничего сверхъестествен...",Neutral,positive,0.469223
1,"Сегодня смотрел этот фильм, если конечно к «эт...",Bad,negative,0.706347
2,Не смог удержатся и не написать лестный отзыв ...,Bad,negative,0.425498
3,"Фильм-история. Сюжет привлекает внимание, это ...",Neutral,neutral,0.600747
4,"Есть одна фраза, высказанная свободным питерск...",Bad,negative,0.742875
...,...,...,...,...
1495,"Действительно, фильм гениальной по сюжету, акт...",Neutral,negative,0.445962
1496,Помоги нам боже пережить этот позор российской...,Bad,negative,0.559731
1497,"Однажды, спасаясь от гангстеров, в маленький а...",Good,negative,0.586232
1498,"- Эдит, ты не можешь это сделать!\n- Как это н...",Good,neutral,0.451288


In [ ]:
# Сохранение в CSV
submission_df.to_csv('submission.csv', index=False)
print("Файл submission.csv создан успешно.")

# дообучить эту гребаную ракетку

In [20]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import pandas as pd
from tqdm import tqdm
from datasets import DatasetDict, Dataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

In [16]:
# Приведение классов к числовому формату
label_map = {"Bad": 0, "Neutral": 1, "Good": 2}

In [17]:
# Подготовка данных для обучения
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=512)

ds = ds.map(lambda x: {'label': [label_map[label] for label in x['label_text']]}, batched=True)
ds = ds.map(tokenize, batched=True)

Map:   0%|          | 0/10500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/10500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [18]:
# Разделение данных
train_dataset = ds['train']
valid_dataset = ds['validation']
test_dataset = ds['test']

In [27]:
# Параметры обучения
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",            # Используем eval_strategy вместо evaluation_strategy
    save_strategy="epoch",            # Сохраняем модель после каждой эпохи, чтобы совпадало с eval_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,      # Загружаем лучшую модель по завершении обучения
    metric_for_best_model="accuracy",
    report_to="none"
)

In [28]:
# Метрики для оценки
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    return {"accuracy": accuracy, "f1": f1, "precision": precision, "recall": recall}

In [29]:
# Обучение модели
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics
)

In [30]:
# Запуск fine-tuning
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.725900,0.794861,0.664000,0.634308,0.660665,0.664000
2,0.594300,0.741426,0.698667,0.694779,0.693494,0.698667
3,0.438000,0.843313,0.704667,0.699141,0.698105,0.704667


TrainOutput(global_step=3939, training_loss=0.6308469656730975, metrics={'train_runtime': 3596.5982, 'train_samples_per_second': 8.758, 'train_steps_per_second': 1.095, 'total_flos': 8289337706496000.0, 'train_loss': 0.6308469656730975, 'epoch': 3.0})

In [31]:
# Оценка на тестовом наборе
predictions, labels, _ = trainer.predict(test_dataset)
predicted_labels = predictions.argmax(axis=1)

In [32]:
# Подсчет метрик на тестовом наборе
accuracy = accuracy_score(labels, predicted_labels)
f1 = f1_score(labels, predicted_labels, average='weighted')
precision = precision_score(labels, predicted_labels, average='weighted')
recall = recall_score(labels, predicted_labels, average='weighted')

print("Test Accuracy:", accuracy)
print("Test F1 Score:", f1)
print("Test Precision:", precision)
print("Test Recall:", recall)

Test Accuracy: 0.6906666666666667
Test F1 Score: 0.6805076374732446
Test Precision: 0.6795958592880142
Test Recall: 0.6906666666666667


In [33]:
# Создание сабмита
label_map_inv = {v: k for k, v in label_map.items()}
submission_df = pd.DataFrame({
    'text': test_dataset['text'],
    'true_label': [label_map_inv[label] for label in labels],
    'predicted_label': [label_map_inv[pred] for pred in predicted_labels]
})

In [34]:
submission_df

,text,true_label,predicted_label
0,"Шла на сеанс, не ожидая ничего сверхъестествен...",Neutral,Good
1,"Сегодня смотрел этот фильм, если конечно к «эт...",Bad,Bad
2,Не смог удержатся и не написать лестный отзыв ...,Bad,Neutral
3,"Фильм-история. Сюжет привлекает внимание, это ...",Neutral,Neutral
4,"Есть одна фраза, высказанная свободным питерск...",Bad,Bad
...,...,...,...
1495,"Действительно, фильм гениальной по сюжету, акт...",Neutral,Bad
1496,Помоги нам боже пережить этот позор российской...,Bad,Bad
1497,"Однажды, спасаясь от гангстеров, в маленький а...",Good,Good
1498,"- Эдит, ты не можешь это сделать!\n- Как это н...",Good,Good


In [ ]:
submission_df.to_csv('submission.csv', index=False)
print("Файл submission.csv создан успешно.")